<a href="https://colab.research.google.com/github/bhardwaj2-6/RAG_Techniques/blob/main/RAG_FROM_SCRATCH/%231_Overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enviornment

In [1]:
!pip install langchain_groq langchain langchain_community langchainhub tiktoken chromadb python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.2 MB/s eta 0:00:0

In [2]:

import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'


In [15]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.4 MB/s eta 0:00:00


# Part 1: Overview

In [16]:
import bs4
import requests
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/90.0.4430.93 Safari/537.36"



In [23]:
### INDEXING ###


# load Documents
loader = WebBaseLoader(
    web_path=("https://medium.com/@nimritakoul01/the-model-context-protocol-mcp-a-complete-tutorial-a3abe8a7f4ef"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name=["h1","p"] # Changed to select all 'h1' (headings) and 'p' (paragraphs) tags.
        )
    )
)
docs=loader.load()

# Verify if docs is empty
if not docs:
    print("Warning: No documents were loaded. Check the web_path and bs_kwargs.")

# split
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits= text_splitter.split_documents(docs)

# Verify if splits is empty
if not splits:
    print("Warning: No splits were created. Check the text_splitter settings.")

# embed
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#persist_directory = "db"
vectorstore = FAISS.from_documents(documents=splits, embedding=model) # Correct Usage: Invoking from_documents() on Chroma Class
retrievers= vectorstore.as_retriever()


###-----------     RETIEVAL AND GENERATION -------------------####

# prompt
prompt= hub.pull("rlm/rag-prompt")

# llm
llm= ChatGroq(model="gemma2-9b-it",api_key=os.environ.get("GROQ_API_KEY"))

# post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# chain
rag_chain = ({"context": retrievers | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser())

# Question
rag_chain.invoke("What is Architecture of  MCP?")



'MCP follows a client-host-server architecture. \nClients are AI applications that want to access external systems, while servers provide standardized access to these systems. This architecture allows for flexible and scalable integration of tools and data sources into AI applications.  \n'